In [22]:
import earthaccess
lon_min = -10
lat_min = 30
lon_max = 35
lat_max = 55

auth = earthaccess.login()
results = earthaccess.search_data(
    short_name="OCO2_L2_Lite_FP",
    temporal=("2024-01-01", "2025-01-02"),
    bounding_box=(-10, 30, 35, 55)
)

In [3]:
print(len(results))
print(results[0].data_links()[0])

461
https://data.gesdisc.earthdata.nasa.gov/data/OCO2_DATA/OCO2_L2_Lite_FP.11.1r/2023/oco2_LtCO2_231231_B11100Ar_240124193942s.nc4


In [4]:
files = earthaccess.download(results[:5], '../data/raw')

QUEUEING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/5 [00:00<?, ?it/s]

In [25]:
import xarray as xr
datasets = [ xr.open_dataset(f'../data/raw/{file.name}', engine = 'netcdf4')
                for file in files]
datasets[0]

<xarray.Dataset> Size: 76MB
Dimensions:                 (sounding_id: 169840, epoch_dimension: 7,
                             vertices: 4, levels: 20, bands: 3, source_files: 16)
Coordinates:
  * sounding_id             (sounding_id) float64 1MB 2.023e+15 ... 2.023e+15
  * vertices                (vertices) int16 8B 1 2 3 4
  * levels                  (levels) int16 40B 1 2 3 4 5 6 ... 15 16 17 18 19 20
  * bands                   (bands) int16 6B 1 2 3
  * source_files            (source_files) object 128B 'oco2_L2StdGL_50513a_2...
Dimensions without coordinates: epoch_dimension
Data variables: (12/20)
    date                    (sounding_id, epoch_dimension) float32 5MB ...
    latitude                (sounding_id) float32 679kB ...
    longitude               (sounding_id) float32 679kB ...
    time                    (sounding_id) datetime64[ns] 1MB ...
    solar_zenith_angle      (sounding_id) float32 679kB ...
    sensor_zenith_angle     (sounding_id) float32 679kB ...
    ...                      ...
    xco2_uncertainty        (sounding_id) float32 679kB ...
    xco2_apriori            (sounding_id) float32 679kB ...
    pressure_levels         (sounding_id, levels) float32 14MB ...
    co2_profile_apriori     (sounding_id, levels) float32 14MB ...
    xco2_averaging_kernel   (sounding_id, levels) float32 14MB ...
    pressure_weight         (sounding_id, levels) float32 14MB ...
Attributes: (12/30)
    filter_function:                   oco2_xco2_quality_flag_b11_1
    bc_function:                       oco2_xco2_bias_correct_b11_1
    l2_reader:                         read_oco_l2_b11
    lite_definition_module:            lite_file_definition_oco2_b11_1
    met_files:                         ['oco2_L2MetGL_50513a_231230_B11008r_2...
    arp_files:                         oco2_ARP_50413_50528_v51_240102223403.h5
    ...                                ...
    Platform:                          OCO-2
    Sensor:                            OCO-2
    L2FullPhysicsExeVersion:           None
    title:                             OCO-2 L2FP Lite Output
    contact:                           Chris O'Dell: Christopher.ODell@colost...
    creation_date:                     Jan 24, 2024

In [27]:
year_from = 2024

combined = xr.concat(datasets, dim='sounding_id', join='outer')

xco2_ds = combined.where(combined["xco2_quality_flag"] == 0, drop=True)
xco2_ds = xco2_ds.where(xco2_ds["time"].dt.year >= year_from, drop=True)
xco2_ds = xco2_ds.where(
        (xco2_ds["latitude"] >= lat_min)
        & (xco2_ds["latitude"] <= lat_max)
        & (xco2_ds["longitude"] >= lon_min)
        & (xco2_ds["longitude"] <= lon_max),
        drop=True,
    )

xco2_df = xco2_ds[["xco2", "time", "latitude", "longitude"]].to_dataframe().reset_index()
xco2_df = xco2_df.rename(columns={"latitude": "lat", "longitude": "lon"})
xco2_df.head()

,sounding_id,xco2,time,lat,lon
0,2.024010e+15,420.261780,2024-01-01 10:44:00.562007040,44.975578,34.747627
1,2.024010e+15,420.460785,2024-01-01 10:44:00.614993920,44.962967,34.769623
2,2.024010e+15,421.595459,2024-01-01 10:44:00.641990144,44.956535,34.780552
3,2.024010e+15,421.978424,2024-01-01 10:44:00.895016448,44.995373,34.740425
4,2.024010e+15,421.174530,2024-01-01 10:44:00.921007104,44.989140,34.751434
